In [ ]:
import csv

# import DataFrame from pandas in order to convert the sql query in a DataFrame
from pandas import DataFrame

# The ipython-sql library is loaded using the %load_ext iPython extension
%load_ext sql

## 1. Make sure ePRTR database is in good shape

TODO: Check db_preconditions!

## 2. Read data from ePRTR database into pandas DataFrame

In [ ]:
year = '2018'

In [ ]:
%%sql sqlite:///data/prtr_en.db sql_query_result << 

SELECT 
    facilities.name AS LPS, 
    activities.prtr_key AS 'PRTR activity',
    nace_code || ': ' || nace_text AS 'NACE',
    activities.prtr_key AS 'GNFR', 
    facilities.administrative_number AS 'E-PRTR/PRTR Facility ID', 
    activities.prtr_key AS 'Height class',
    facilities.wgs84_x AS 'Longitude (deg)', 
    facilities.wgs84_y AS 'Latitude (deg)',

    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Nitrogen oxides (NOx/NO2)') AS "NOx (as NO2) (kt)",
    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Non-methane volatile organic compounds (NMVOC)') 
    AS "NMVOC (kt)",
    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Sulphur oxides (SOx/SO2)') AS "SOx (as SO2) (kt)",
    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Ammonia (NH3)') AS "NH3 (kt)",
    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Particulate matter (PM2.5)') AS "PM2.5 (kt)",
    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Particulate matter (PM10)') AS "PM10 (kt)",
    SUM(releases.annual_load/1000000) FILTER (WHERE substance_name = 'Carbon monoxide (CO)') AS "CO (kt)",
    SUM(releases.annual_load/1000) FILTER (WHERE substance_name = 'Lead and compounds (as Pb)') AS "Pb (t)",
    SUM(releases.annual_load/1000) FILTER (WHERE substance_name = 'Cadmium and compounds (as Cd)') AS "Cd (t)",
    SUM(releases.annual_load/1000) FILTER (WHERE substance_name = 'Mercury and compounds (as Hg)') AS "Hg (t)",
    SUM(releases.annual_load*1000) FILTER (WHERE substance_name = 'PCDD + PCDF (dioxins + furans)(as Teq)') 
    AS "PCDD/ PCDF (dioxins/ furans)(g I-Teq)",
    SUM(releases.annual_load/1000) FILTER (WHERE substance_name = 'Polycyclic aromatic hydrocarbons (PAHs)') AS "PAHs (t)",
    SUM(releases.annual_load) FILTER (WHERE substance_name = 'Hexachlorobenzene (HCB)') AS "HCB (kg)",
    SUM(releases.annual_load) FILTER (WHERE substance_name = 'Polychlorinated biphenyls') AS "PCBs (kg)"

FROM facilities
    JOIN releases ON facilities.id = releases.facility_ID
    JOIN activities ON releases.facility_ID = activities.facility_ID

WHERE facilities.year = :year AND releases.compartment = 'Air' AND activities.main_activity = 1
GROUP BY facilities.id
ORDER BY name

In [ ]:
data = sql_query_result.DataFrame()
data.info()
data

## 3. Fill GNFR and stack height columns with proper values derived from prtr activities

In [ ]:
gnfr_mapping = {}
chimney_mapping={}
with open('data/gnfr_mapping.csv', newline='') as mapping_file: 
    for line in csv.reader(mapping_file, delimiter=';'): 
        gnfr_mapping[line[0]] = line[2]
        chimney_mapping[line[0]] = line[3]

data.replace({'GNFR': gnfr_mapping}, inplace=True) # GNFR sector mapping
data.replace({'Height class': chimney_mapping}, inplace=True) # Stack height mapping

# Make sure all point sources have GNFR and stack height set
assert len(data[data['GNFR'] == '']) == 0, f"There are still {len(data[data['GNFR'] == ''])} point sources with GNFR listed as '<empty>'!"
assert len(data[~data['GNFR'].str.contains('_')]) == 0, f"There are still {len(data[~data['GNFR'].str.contains('_')])} point sources with GNFR listed as '<prtr key>'!"
assert len(data[~data['Height class'].str.isnumeric()]) == 0, f"There are still {len(data[~data['Height class'].str.isnumeric()])} point sources with an invalid height class!"

## 4. QA

In [ ]:
# Make sure LPS name, GNFR, ID, height class, longitude and latitude are set for all point sources
assert len(data[data['LPS'] == '']) == 0, f"There are {len(data[data['LPS'] == ''])} point source(s) without a name!"
assert len(data[data['GNFR'] == '']) == 0, f"There are {len(data[data['GNFR'] == ''])} point source(s) without a GNFR sector!"
assert len(data[data['E-PRTR/PRTR Facility ID'] == '']) == 0, f"There are {len(data[data['E-PRTR/PRTR Facility ID'] == ''])} point source(s) without an ID!"
assert len(data[data['Height class'] == '']) == 0, f"There are {len(data[data['Height class'] == ''])} point source(s) without a height class!"
assert len(data[data['Longitude (deg)'] == '']) == 0, f"There are {len(data[data['Longitude (deg)'] == ''])} point source(s) without a longitude value!"
assert len(data[data['Latitude (deg)'] == '']) == 0, f"There are {len(data[data['Latitude (deg)'] == ''])} point source(s) without a latitude value!"

In [ ]:
# Check total emissions
sum = DataFrame({"Sum LPS": data.iloc[:, 8:23].sum()})
sum["National Total"] = [1210.50, 1125.02, 291.78, 601.24, 94.66, 207.01, 2957.80, 162.18, 11.94, 8.30, 119.8, 75.71, 12.82, 219.33]
sum["Percentage"] = sum["Sum LPS"] / sum["National Total"] * 100
sum["Problem?"] = sum["Sum LPS"] >= sum["National Total"]
sum

In [ ]:
# Check outliers
data.boxplot(column = ['NOx (as NO2) (kt)', 'NMVOC (kt)', 'SOx (as SO2) (kt)', 'NH3 (kt)'], figsize=(20, 10))

In [ ]:
data[data['NOx (as NO2) (kt)'] > data['NOx (as NO2) (kt)'].quantile(0.98)][['LPS', 'NACE', 'GNFR', 'Height class', 'Longitude (deg)', 'Latitude (deg)', 'NOx (as NO2) (kt)']].sort_values(by=['NOx (as NO2) (kt)'], ascending=False)

In [ ]:
data[data['NMVOC (kt)'] > data['NMVOC (kt)'].quantile(0.9)][['LPS', 'NACE', 'GNFR', 'Height class', 'Longitude (deg)', 'Latitude (deg)', 'NMVOC (kt)']].sort_values(by=['NMVOC (kt)'], ascending=False)

In [ ]:
data[data['SOx (as SO2) (kt)'] > data['SOx (as SO2) (kt)'].quantile(0.95)][['LPS', 'NACE', 'GNFR', 'Height class', 'Longitude (deg)', 'Latitude (deg)', 'SOx (as SO2) (kt)']].sort_values(by=['SOx (as SO2) (kt)'], ascending=False)

In [ ]:
data[data['NH3 (kt)'] > data['NH3 (kt)'].quantile(0.98)][['LPS', 'NACE', 'GNFR', 'Height class', 'Longitude (deg)', 'Latitude (deg)', 'NH3 (kt)']].sort_values(by=['NH3 (kt)'], ascending=False)

In [ ]:
data.boxplot(column = ['PM10 (kt)', 'CO (kt)', 'Pb (t)', 'Cd (t)', 'Hg (t)', 'PAHs (t)'], figsize=(20, 10))

In [ ]:
data[data['CO (kt)'] > data['CO (kt)'].quantile(0.9)][['LPS', 'NACE', 'GNFR', 'Height class', 'Longitude (deg)', 'Latitude (deg)', 'CO (kt)']].sort_values(by=['CO (kt)'], ascending=False)

In [ ]:
data[data['Pb (t)'] > data['Pb (t)'].quantile(0.9)][['LPS', 'NACE', 'GNFR', 'Height class', 'Longitude (deg)', 'Latitude (deg)', 'Pb (t)']].sort_values(by=['Pb (t)'], ascending=False)

In [ ]:
# Find duplicate coordinates (NECD review finding)
data['QA: Duplicate coordinates'] = data[['Longitude (deg)', 'Latitude (deg)']].duplicated(keep=False)
data[data['QA: Duplicate coordinates'] == True].sort_values(by=['Longitude (deg)'])

In [ ]:
# Unique key from LPS name, GNFR and stack height (NECD review finding)
data['QA: Duplicate key'] = data[['LPS', 'GNFR', 'Height class']].duplicated(keep=False)
data[data['QA: Duplicate key'] == True].sort_values(by=['LPS'])

## 5. Write data to CSV file to be copied to Excel CLRTAP template

In [ ]:
data.to_csv('output/Convert ePRTR data to CLRTAP LPS.csv', index=False, sep='|', decimal=',', columns=['LPS', 'GNFR', 'E-PRTR/PRTR Facility ID', 'Height class', 'Longitude (deg)', 'Latitude (deg)', 'NOx (as NO2) (kt)', 'NMVOC (kt)', 'SOx (as SO2) (kt)', 'NH3 (kt)', 'PM2.5 (kt)', 'PM10 (kt)', 'CO (kt)', 'Pb (t)', 'Cd (t)', 'Hg (t)', 'PCDD/ PCDF (dioxins/ furans)(g I-Teq)', 'PAHs (t)', 'HCB (kg)', 'PCBs (kg)'])

## Optional: Create shapefile and inline map to visualize data

In [ ]:
import geopandas

locations = geopandas.GeoDataFrame(data, geometry=geopandas.points_from_xy(data['Longitude (deg)'], data['Latitude (deg)']), crs='epsg:4326')
locations.to_file('output/geo/Convert ePRTR data to CLRTAP LPS.shp', encoding='utf8')

germany = geopandas.read_file('data/geo/germany.geo.json').plot(figsize=(15, 15), alpha=0.3, edgecolor='k')
locations.plot("GNFR", ax=germany, legend=True, figsize=(15, 15), alpha=0.7)